# Pandas: Time Series analizė ir DateTime operacijos

Faile pateikiami aiškūs ir nuoseklūs pavyzdžiai, kaip su `pandas` dirbti su laiko duomenimis:

- `datetime64` duomenų tipas
- `to_datetime()` funkcija
- datų formatavimas ir kodai
- datų komponentų išgavimas
- laiko intervalai (`timedelta`)
- trūkstami laiko duomenys
- laiko eilučių agregavimas ir resampling
- slenkantys (rolling) skaičiavimai

Pavyzdžiai orientuoti į duomenų analitikos kontekstą (pardavimų dinamika laike).
Komentarai pateikiami techniniu, aiškinamuoju stiliumi.


In [2]:
import pandas as pd
import numpy as np

pd.__version__


'2.3.3'

## 1. Pavyzdiniai laiko duomenys

Naudojami paprasti dieniniai pardavimų duomenys su tarpais laike.
Tokie duomenys dažni realiuose verslo scenarijuose.


In [3]:
data = {
    "date": [
        "2025-01-01", "2025-01-02", "2025-01-03",
        "2025-01-05", "2025-01-06",
        "2025-01-09", "2025-01-10"
    ],
    "revenue": [1200, 1300, 1250, 1400, 1500, 1600, 1550]
}

df = pd.DataFrame(data)
df


,date,revenue
0,2025-01-01,1200
1,2025-01-02,1300
2,2025-01-03,1250
3,2025-01-05,1400
4,2025-01-06,1500
5,2025-01-09,1600
6,2025-01-10,1550


## 2. DateTime duomenų tipas (`datetime64`)

Pagal nutylėjimą datos dažnai būna `object` tipo.
Analizei jos turi būti konvertuojamos į `datetime64`.

Dažna klaida – palikti datas kaip tekstą ir vėliau bandyti naudoti `.dt` funkcijas.


In [4]:
df.dtypes


date       object
revenue     int64
dtype: object

In [5]:
# Konvertavimas į datetime64
df["date"] = pd.to_datetime(df["date"])
df.dtypes


date       datetime64[ns]
revenue             int64
dtype: object

## 3. `to_datetime()` ir dažnos klaidos

`to_datetime()` automatiškai atpažįsta daugumą formatų, tačiau:
- neteisingas formatas sukelia `NaT`
- neaiškūs simboliai turi būti tvarkomi iš anksto

Geroji praktika – naudoti `errors='coerce'` ir pasitikrinti rezultatą.


In [8]:
# Pavyzdys su klaida duomenyse
dates_raw = pd.Series(["2025-01-01", "2025-13-01", "2025-01-03"])

parsed = pd.to_datetime(dates_raw, errors="coerce")
parsed


0   2025-01-01
1          NaT
2   2025-01-03
dtype: datetime64[ns]

## 4. Datų formatavimas

Datų atvaizdavimas nekeičia pačių duomenų, keičia tik išvaizdą.
Tai svarbu ataskaitose ir eksportuojant duomenis.

Dažna klaida – manyti, kad `strftime()` pakeičia datų tipą.


In [9]:
df["date_str"] = df["date"].dt.strftime("%Y-%m-%d")
df


,date,revenue,date_str
0,2025-01-01,1200,2025-01-01
1,2025-01-02,1300,2025-01-02
2,2025-01-03,1250,2025-01-03
3,2025-01-05,1400,2025-01-05
4,2025-01-06,1500,2025-01-06
5,2025-01-09,1600,2025-01-09
6,2025-01-10,1550,2025-01-10


In [10]:
df.dtypes

date        datetime64[ns]
revenue              int64
date_str            object
dtype: object

### Dažniausi formatavimo kodai

- `%Y` – metai
- `%m` – mėnuo (01–12)
- `%d` – diena
- `%b` – trumpas mėnesio pavadinimas
- `%B` – pilnas mėnesio pavadinimas


## 5. Datų komponentų išgavimas

Iš `datetime64` galima išgauti atskiras laiko dalis.
Tai leidžia lengvai atlikti periodines analizes.


In [11]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.day_name()

df


,date,revenue,date_str,year,month,day,weekday
0,2025-01-01,1200,2025-01-01,2025,1,1,Wednesday
1,2025-01-02,1300,2025-01-02,2025,1,2,Thursday
2,2025-01-03,1250,2025-01-03,2025,1,3,Friday
3,2025-01-05,1400,2025-01-05,2025,1,5,Sunday
4,2025-01-06,1500,2025-01-06,2025,1,6,Monday
5,2025-01-09,1600,2025-01-09,2025,1,9,Thursday
6,2025-01-10,1550,2025-01-10,2025,1,10,Friday


## 6. Laiko intervalai (`timedelta`)

`Timedelta` aprašo laiko skirtumą tarp datų.
Naudojama skirtumams, trukmėms ir vėlavimams skaičiuoti.


In [14]:
df["prev_date"] = df["date"].shift(1)
df["days_since_prev"] = df["date"] - df["prev_date"]

df


,date,revenue,date_str,year,month,day,weekday,prev_date,days_since_prev
0,2025-01-01,1200,2025-01-01,2025,1,1,Wednesday,NaT,NaT
1,2025-01-02,1300,2025-01-02,2025,1,2,Thursday,2025-01-01,1 days
2,2025-01-03,1250,2025-01-03,2025,1,3,Friday,2025-01-02,1 days
3,2025-01-05,1400,2025-01-05,2025,1,5,Sunday,2025-01-03,2 days
4,2025-01-06,1500,2025-01-06,2025,1,6,Monday,2025-01-05,1 days
5,2025-01-09,1600,2025-01-09,2025,1,9,Thursday,2025-01-06,3 days
6,2025-01-10,1550,2025-01-10,2025,1,10,Friday,2025-01-09,1 days


## 7. Laiko indeksas ir trūkstami duomenys

Time series analizėje dažnai reikia pilnos laiko sekos.
Tam naudojamas laiko indeksas ir `asfreq()`.


In [15]:
# Nustatomas laiko indeksas
ts = df.set_index("date")[["revenue"]]

# Sukuriama pilna dienų seka
ts_daily = ts.asfreq("D")
ts_daily


,revenue
date,
2025-01-01,1200.0
2025-01-02,1300.0
2025-01-03,1250.0
2025-01-04,NaN
2025-01-05,1400.0
2025-01-06,1500.0
2025-01-07,NaN
2025-01-08,NaN
2025-01-09,1600.0


Dažna klaida – ignoruoti trūkstamas datas, dėl ko iškraipomi vidurkiai ir slenkantys skaičiavimai.

Geroji praktika – aiškiai nuspręsti, kaip trūkstami duomenys turi būti tvarkomi.


In [16]:
# Paprastas trūkstamų reikšmių tvarkymas
ts_daily["revenue_filled"] = ts_daily["revenue"].fillna(0)
ts_daily


,revenue,revenue_filled
date,,
2025-01-01,1200.0,1200.0
2025-01-02,1300.0,1300.0
2025-01-03,1250.0,1250.0
2025-01-04,NaN,0.0
2025-01-05,1400.0,1400.0
2025-01-06,1500.0,1500.0
2025-01-07,NaN,0.0
2025-01-08,NaN,0.0
2025-01-09,1600.0,1600.0


## 8. Agregavimas ir resampling

Resampling leidžia pakeisti laiko dažnį:
- diena → savaitė
- diena → mėnuo

Tai viena svarbiausių time series analitikos operacijų.


In [17]:
# Savaitinis agregavimas
weekly = ts_daily["revenue"].resample("W").sum()
weekly


date
2025-01-05    5150.0
2025-01-12    4650.0
Freq: W-SUN, Name: revenue, dtype: float64

Dažna klaida – bandyti naudoti `resample()` be laiko indekso.
`resample()` veikia tik su `DatetimeIndex`.


In [18]:
# Mėnesinis vidurkis
monthly_avg = ts_daily["revenue"].resample("M").mean()
monthly_avg


C:\Users\User\AppData\Local\Temp\ipykernel_29080\1457564949.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_avg = ts_daily["revenue"].resample("M").mean()


date
2025-01-31    1400.0
Freq: ME, Name: revenue, dtype: float64

## 9. Rolling (slenkančios) agregacijos

Rolling agregacijos naudojamos tendencijoms išlyginti.
Tai dažna praktika finansuose ir pardavimų analizėje.


In [19]:
# 3 dienų slenkantis vidurkis
ts_daily["rolling_3d_avg"] = ts_daily["revenue"].rolling(window=3).mean()
ts_daily


,revenue,revenue_filled,rolling_3d_avg
date,,,
2025-01-01,1200.0,1200.0,NaN
2025-01-02,1300.0,1300.0,NaN
2025-01-03,1250.0,1250.0,1250.0
2025-01-04,NaN,0.0,NaN
2025-01-05,1400.0,1400.0,NaN
2025-01-06,1500.0,1500.0,NaN
2025-01-07,NaN,0.0,NaN
2025-01-08,NaN,0.0,NaN
2025-01-09,1600.0,1600.0,NaN


Dažna klaida – pamiršti, kad pirmos reikšmės bus `NaN`, nes langui dar trūksta duomenų.


## 10. Dažnos klaidos ir gerosios praktikos (santrauka)

Dažnos klaidos:
- palikti datas kaip `object`
- naudoti `.dt` metodus be `datetime64`
- naudoti `resample()` be laiko indekso
- ignoruoti trūkstamas datas
- neteisingai interpretuoti `NaT`

Gerosios praktikos:
- visada konvertuoti datas su `to_datetime()`
- naudoti `errors='coerce'` nešvariems duomenims
- naudoti laiko indeksą time series analizei
- aiškiai tvarkyti trūkstamus laikotarpius
- naudoti `rolling()` tik su prasmingu lango dydžiu
